In [44]:
from functools import wraps
import inspect

In [2]:
def savings(cls):
    cls.account_type = "savings"
    return cls

def checking(cls):
    cls.account_type = "checking"
    return cls

In [4]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Сhecking(Account):
    pass

@checking
class Bank2Сhecking(Account):
    pass

In [6]:
Bank2Сhecking.__dict__, Bank1Savings.__dict__

(mappingproxy({'__module__': '__main__',
               '__doc__': None,
               'account_type': 'checking'}),
 mappingproxy({'__module__': '__main__',
               '__doc__': None,
               'account_type': 'savings'}))

let's do parametrized decorator

In [7]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [8]:
@account_type("savings")
class Bank1Savings(Account):
    pass

@account_type("checking")
class Bank1Сhecking(Account):
    pass


In [9]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'savings'})

let's create a decorator to inject a new function into the class before we return it

In [10]:
def hello(cls):
    cls.hello = lambda self: f"{self} says hello!"
    return cls

In [18]:
@hello
class Person:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [20]:
for name, obj in vars(Person).items():
        if callable(obj):
            print("decorating:",  name)


decorating: __init__
decorating: __str__
decorating: hello


In [12]:
p = Person("alex")
p.hello()

'alex says hello!'

In [14]:
def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f"Log {fn.__qualname__}({args}{kwargs}) = {result}")
        return result
    return inner

In [15]:
class Person:
    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age
    @func_logger   
    def greet(self):
        return f"Hello! my name is {self.name}"

In [16]:
p = Person("hun", 34)

Log Person.__init__((<__main__.Person object at 0x0000023C801D06C8>, 'hun', 34){}) = None


In [17]:
p.greet()

Log Person.greet((<__main__.Person object at 0x0000023C801D06C8>,){}) = Hello! my name is hun


'Hello! my name is hun'

let's create a class decorator that decorates any callable in a given class

In [25]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating:", cls, name)
            setattr(cls, name, func_logger(obj))
    return cls


In [26]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def greet(self):
        return f"Hello! my name is {self.name}"

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [27]:
p = Person("John", 54)
p.greet()

Log Person.__init__((<__main__.Person object at 0x0000023C801D6688>, 'John', 54){}) = None
Log Person.greet((<__main__.Person object at 0x0000023C801D6688>,){}) = Hello! my name is John


'Hello! my name is John'

static and class methods are not callables

In [28]:
@class_logger
class Person:
    
    @staticmethod
    def static_method():
        print("static method invoked...")
        
    @classmethod
    def cls_method(cls):
        print(f"cls_method invoked for {cls}...")
        
    def instance_method(self):
        print(f"instance_method invoked for {self}...")

decorating: <class '__main__.Person'> instance_method


In [29]:
vars(Person)

mappingproxy({'__module__': '__main__',
              'static_method': <staticmethod at 0x23c801bdcc8>,
              'cls_method': <classmethod at 0x23c801bd948>,
              'instance_method': <function __main__.Person.instance_method(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

we need to decorate static_method and cls_method before @staticmethod and @classmethod decorators

In [30]:
class Person:
    
    @staticmethod
    @func_logger
    def static_method():
        print("static method invoked...")
        
    @classmethod
    @func_logger 
    def cls_method(cls):
        print(f"cls_method invoked for {cls}...")

how to modify our class decorator so it would work with @staticmethod and @classmethod

In [31]:
Person.__dict__['static_method'].__func__

<function __main__.Person.static_method()>

In [32]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating callable:", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print("decorating static method:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print("decorating class method:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
    return cls


In [33]:
@class_logger
class Person:
    
    @staticmethod
    def static_method():
        print("static method invoked...")
        
    @classmethod
    def cls_method(cls):
        print(f"cls_method invoked for {cls}...")
        
    def instance_method(self):
        print(f"instance_method invoked for {self}...")

decorating static method: <class '__main__.Person'> static_method
decorating class method: <class '__main__.Person'> cls_method
decorating callable: <class '__main__.Person'> instance_method


#### what about properties?

In [34]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property #class_logger won't decorate property
    def name(self):
        return self._name

decorating callable: <class '__main__.Person'> __init__


In [35]:
isinstance(Person.__dict__['name'], property)

True

In [36]:
prop = Person.__dict__['name']
prop.fget

<function __main__.Person.name(self)>

In [37]:
prop.fset, prop.fdel

(None, None)

maybe we can decorate fget, fset, fdel which are not None. we can create a new property

In [38]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print("decorating callable:", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
        elif isinstance(obj, staticmethod):
            print("decorating static method:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print("decorating class method:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print("decorating property:", cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
                
    return cls


In [39]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property 
    def name(self):
        return self._name

decorating callable: <class '__main__.Person'> __init__
decorating property: <class '__main__.Person'> name


In [40]:
p = Person("David")
p.name

Log Person.__init__((<__main__.Person object at 0x0000023C801E9188>, 'David'){}) = None
Log Person.name((<__main__.Person object at 0x0000023C801E9188>,){}) = David


'David'

not every callable is a function that can be decorated

In [41]:
@class_logger
class Person:
    class Other:
        def __call__(self):
            print("called instance of Other")
    other = Other()
        

decorating callable: <class '__main__.Person'> Other
decorating callable: <class '__main__.Person'> other


In [42]:
Person.Other # it's now a function, not a class, and it's a problem

<function __main__.Person.Other()>

In [43]:
Person.other # we don't have instance of Other, we have a function

<function __main__.func_logger.<locals>.inner()>

we can use inspect module to restrict our callable thing

let's see how inspect works

In [45]:
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

In [46]:
keys = ('static_method', 'cls_method', 'inst_method', 'name', '__add__', 'Other', 'other')
inspect_funcs = ('isroutine', 'ismethod', 'isfunction', 'isbuiltin', 'ismethoddescriptor')

In [47]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


In [48]:
help(inspect)

Help on module inspect:

NAME
    inspect - Get useful information from live Python objects.

DESCRIPTION
    This module encapsulates the interface provided by the internal special
    attributes (co_*, im_*, tb_*, etc.) in a friendlier fashion.
    It also provides some help for examining source code and class layout.
    
    Here are some of the useful functions provided by this module:
    
        ismodule(), isclass(), ismethod(), isfunction(), isgeneratorfunction(),
            isgenerator(), istraceback(), isframe(), iscode(), isbuiltin(),
            isroutine() - check object types
        getmembers() - get members of an object that satisfy a given condition
    
        getfile(), getsourcefile(), getsource() - find an object's source code
        getdoc(), getcomments() - get documentation on an object
        getmodule() - determine the module that an object came from
        getclasstree() - arrange classes so as to represent their hierarchy
    
        getargvalues(),

In [49]:
help(inspect.ismethod)

Help on function ismethod in module inspect:

ismethod(object)
    Return true if the object is an instance method.
    
    Instance method objects provide these attributes:
        __doc__         documentation string
        __name__        name with which this method was defined
        __func__        function object containing implementation of method
        __self__        instance to which this method is bound



In [53]:
help(inspect.isroutine)

Help on function isroutine in module inspect:

isroutine(object)
    Return true if the object is any kind of function or method.



In [54]:
help(inspect.iscode)

Help on function iscode in module inspect:

iscode(object)
    Return true if the object is a code object.
    
    Code objects provide these attributes:
        co_argcount         number of arguments (not including *, ** args
                            or keyword only arguments)
        co_code             string of raw compiled bytecode
        co_cellvars         tuple of names of cell variables
        co_consts           tuple of constants used in the bytecode
        co_filename         name of file in which this code object was created
        co_firstlineno      number of first line in Python source code
        co_flags            bitmap: 1=optimized | 2=newlocals | 4=*arg | 8=**arg
                            | 16=nested | 32=generator | 64=nofree | 128=coroutine
                            | 256=iterable_coroutine | 512=async_generator
        co_freevars         tuple of names of free variables
        co_kwonlyargcount   number of keyword only arguments (not including *

In [50]:
inspect.ismethod(MyClass.inst_method)

False

In [51]:
p = MyClass()
inspect.ismethod(p.inst_method)

True

In [52]:
inspect.isroutine(MyClass.inst_method) 

True

let's modify our logger - cause 'callable' is too broad

In [55]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        
        if isinstance(obj, staticmethod):
            print("decorating static method:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            print("decorating class method:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print("decorating property:", cls, name)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print("decorating callable:", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
                
    return cls


In [56]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    @class_logger
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

decorating callable: <class '__main__.MyClass.Other'> __call__
decorating static method: <class '__main__.MyClass'> static_method
decorating class method: <class '__main__.MyClass'> cls_method
decorating callable: <class '__main__.MyClass'> inst_method
decorating property: <class '__main__.MyClass'> name
decorating callable: <class '__main__.MyClass'> __add__


let's clean up our logger

In [61]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        
        if isinstance(obj, staticmethod) or isinstance(obj, classmethod):
            type_ = type(obj)
            print(f"decorating {type_.__name__}:", cls, name)
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        
        elif isinstance(obj, property):
            print("decorating property:", cls, name)
            methods = (("fget", "getter"), ("fset", "setter"), ("fdel", "deleter"),)
            for prop, method in methods:
                if getattr(obj, prop):
                    obj = getattr(obj,method)(func_logger(getattr(obj, prop)))
            
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print("decorating callable:", cls, name)
            original_func = obj
            decorated_func = func_logger(original_func)
            setattr(cls, name, decorated_func)
                
    return cls


In [62]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    @class_logger
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

decorating callable: <class '__main__.MyClass.Other'> __call__
decorating staticmethod: <class '__main__.MyClass'> static_method
decorating classmethod: <class '__main__.MyClass'> cls_method
decorating callable: <class '__main__.MyClass'> inst_method
decorating property: <class '__main__.MyClass'> name
decorating callable: <class '__main__.MyClass'> __add__
